# Peru: Two Years of COVID-19
A very simple analysis of the impact of COVID-19 in Peru with Pandas, Geopandas and Matplotlib in Python using Jupiter Notebooks

### How hard has COVID-19 struck a city / country / region?

One way to address this question is to take a look at the deaths, or more precisely, the excess mortality caused by the virus.

In this occassion, we desire to make a very simple analysis of the effects of COVID-19 in each "departamento" (sort of equivalent to a U.S. State) of Peru.

In order to do that, we are going to create **choropleth maps** with monthly deaths (adjusted by population) ocurred in a given month since 2020-01 for each "departamento", and display these maps through a .gif.

Now, for the sake of simplicity, we are going to work with total and not excess deaths and assume that "DEPARTAMENTO DOMICILIO" stands for the place ("departamento") where the decease took place. In reality, "DEPARTAMENTO DOMICILIO" accounts for the last informed "departamento" or place of residency.

We are going to work with the national deaths database of Peru, SINADEF, and the GeoJSON data of this country, available at GitHub.

In [1]:
# Import dependencies
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import urllib
import subprocess
import glob

Read dataset and store it in "df" `dataframe`.

In [2]:
df=pd.read_csv("fallecidos_sinadef.csv",engine="python",encoding='utf-8',sep="|")

In [3]:
df.head()

,Nº,TIPO SEGURO,SEXO,EDAD,TIEMPO EDAD,ESTADO CIVIL,NIVEL DE INSTRUCCIÓN,ETNIA,COD# UBIGEO DOMICILIO,PAIS DOMICILIO,...,DEBIDO A (CAUSA B),CAUSA B (CIE-X),DEBIDO A (CAUSA C),CAUSA C (CIE-X),DEBIDO A (CAUSA D),CAUSA D (CIE-X),DEBIDO A (CAUSA E),CAUSA E (CIE-X),DEBIDO A (CAUSA F),CAUSA F (CIE-X)
0,1,IGNORADO,FEMENINO,64,AÑOS,SOLTERO,IGNORADO,MESTIZO,92-33-24-01-01-000,PERU,...,INFARTO RECIENTE Y ANTIGUO DE MIOCARDIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,SIS,FEMENINO,15,MINUTOS,SOLTERO,SUPERIOR NO UNIV. COMP.,MESTIZO,92-33-12-08-06-000,PERU,...,DIFICULTAD RESPIRATORIA DEL RECIEN NACIDO,P229,INMATURIDAD EXTREMA,P072,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
2,3,ESSALUD,MASCULINO,97,AÑOS,CASADO,PRIMARIA INCOMPLETA,MESTIZO,92-33-04-01-23-000,PERU,...,ENFERMEDAD RENAL,N189,ENFERMEDAD PULMONAR INTERSTICIAL DIFUSA,J849,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
3,4,IGNORADO,MASCULINO,31,AÑOS,SOLTERO,IGNORADO,MESTIZO,92-33-07-06-01-000,PERU,...,EDEMA PULMONAR,J81X,EN INVESTIGACION,R99X,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
4,5,IGNORADO,MASCULINO,59,AÑOS,SOLTERO,IGNORADO,MESTIZO,92-33-24-01-01-000,PERU,...,SHOCK HIPOVOLEMICO,SIN REGISTRO,SUCESO DE TRANSITO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO


In [4]:
print(df.shape)
print(df.columns)

(846732, 32)
Index(['Nº', 'TIPO SEGURO', 'SEXO', 'EDAD', 'TIEMPO EDAD', 'ESTADO CIVIL',
       'NIVEL DE INSTRUCCIÓN', 'ETNIA', 'COD# UBIGEO DOMICILIO',
       'PAIS DOMICILIO', 'DEPARTAMENTO DOMICILIO', 'PROVINCIA DOMICILIO',
       'DISTRITO DOMICILIO', 'FECHA', 'AÑO', 'MES', 'TIPO LUGAR',
       'INSTITUCION', 'MUERTE VIOLENTA', 'NECROPSIA', 'DEBIDO A (CAUSA A)',
       'CAUSA A (CIE-X)', 'DEBIDO A (CAUSA B)', 'CAUSA B (CIE-X)',
       'DEBIDO A (CAUSA C)', 'CAUSA C (CIE-X)', 'DEBIDO A (CAUSA D)',
       'CAUSA D (CIE-X)', 'DEBIDO A (CAUSA E)', 'CAUSA E (CIE-X)',
       'DEBIDO A (CAUSA F)', 'CAUSA F (CIE-X)'],
      dtype='object')


- We are interested in "FECHA" `string column` and "DEPARTAMENTO DOMICILIO" `string column`. In other words, when and where a death took place. 
- We also want to order this dataset by date, from older to newer, so we create a new column "DATE" `datetime column` based on "FECHA" `string column` and apply the `sort_values` method.
- Inspecting "FECHA" `string column` we realise that it goes back to 2017 and up to frebrueary 2022. Since covid-19 pandemic struck Peru in March 2020, we filter the dataset on "AÑO" `int column` >= 2020.
- Since "DEPARTAMENTO DOMICILIO" `string column` includes places outside of Peru, we filter the dataset on "PAIS DOMICILIO" `string column` == "PERU".
- There is an empty string in "DEPARTAMENTO DOMICILIO" `string column` representing the unknown, so we get rid of it by applying yet another filter.
- Now that our dataset is ordered by date, we create a new column "MES-AÑO" `string column` based of the year and month of "FECHA" `string column`, as in, 2020-01, 2020-02, and so on.
- Finally, we create a subset of our dataset with only three columns: "DEPARTAMENTO DOMICILIO" `string column`, "MES-AÑO" `string column` and "SEXO" `string column`. We could have picked any other column instead of "SEXO" as long as it holds one entry per row, which most columns do. This is beceause we are going to group our dataset by "MES-AÑO" `string column` and **count** occurrences of the third column (deaths).

In [5]:
df['DATE'] = df['FECHA'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df = df.sort_values(by=['DATE'], ascending=True)
df=df[(df["PAIS DOMICILIO"]=="PERU") & (df["AÑO"].isin([2020,2021,2022]))]
df["DEPARTAMENTO DOMICILIO"] = df["DEPARTAMENTO DOMICILIO"].map(str.strip)
df=df[df["DEPARTAMENTO DOMICILIO"]!=""]
df["MES-AÑO"]=df["FECHA"].apply(lambda x: x[:5])+df["FECHA"].apply(lambda x: x[5:7])
df=df[['DEPARTAMENTO DOMICILIO',"MES-AÑO","SEXO"]]

Now that we have a more concise dataset, we can apply two `for loop`s in order to obtain a list of dicts, where each dict represents a "departamento" ("DEPARTAMENTO DOMICILIO" `string column`) and is going to be comprised of "date", "deaths" key-value pairs.

In [6]:
result=[]
max=0
for dep in df["DEPARTAMENTO DOMICILIO"].unique():
  data={}
  sdf=df[df["DEPARTAMENTO DOMICILIO"]==dep]
  sdf=sdf.groupby(sdf['MES-AÑO']).count().reset_index()
  for date in df['MES-AÑO'].unique():
    try:
      data[date]=sdf[sdf["MES-AÑO"]==date]["SEXO"].values[0]
    except IndexError:
      data[date]=0
    if data[date]>max:
      max=data[date]
  result.append(data)

Now that we have a list of dicts, we can easily convert it into a pandas dataframe object and store it in "final_df" `dataframe`, setting each "departamento" as an index. Then, we sort this brand new dataframe by its index in alphabetical order so we can match it with the new dataset (GeoJSON) that we are going to import afterwards.

In [7]:
final_df=pd.DataFrame(result,index=df["DEPARTAMENTO DOMICILIO"].unique()) 
final_df.sort_index(inplace=True)

If we inspect this newly created dataset, we can appreciate that it contains 25 "departamento"s (places where deaths occurred) and 27 "MES-AÑO" (months).

In [8]:
final_df.shape

(25, 27)

- A proper comparison of deaths in each "departamento" shall take into account how many people live in such "departamento". So, in the end, we are going to measure deaths adjusted by population.
- In order to do that we need the population of each "departamento", data that can be retrieved directly from [here](https://es.wikipedia.org/wiki/Anexo:Departamentos_del_Perú_por_población).
- This wikipedia dataset can be read directly with pandas `read_html` method but, first, we need to parse the last part of the url for it contains accents.
- Then, we store that population dataset in "pop_df" `dataframe`.

In [44]:
base_url="https://es.wikipedia.org/wiki/"
query='Anexo:Departamentos_del_Perú_por_población'
query=urllib.parse.quote(query)
url=base_url+query
url
pop_df=pd.read_html(url)[0]

In [45]:
pop_df

Ubigeo        Departamento           Capital Superficie(km²)  Población  \
   Ubigeo        Departamento           Capital Superficie(km²) Censo 2017   
0       1            Amazonas       Chachapoyas       39 249,13    379 384   
1       2              Áncash            Huaraz       35 914,81  1 083 519   
2       3            Apurímac           Abancay       20 895,79    405 759   
3       4            Arequipa          Arequipa       63 345,39  1 382 730   
4       5            Ayacucho          Ayacucho       43 814,80    616 176   
5       6           Cajamarca         Cajamarca       33 317,54  1 341 012   
6       7          Callao[4]​            Callao           14698    994 494   
7       8               Cusco             Cusco       71 986,50  1 205 527   
8       9        Huancavelica      Huancavelica       22 131,47    347 639   
9      10             Huánuco           Huánuco       36 848,85    721 047   
10     11                 Ica               Ica       21 327,83    850 765   
11     12               Junín          Huancayo       44 197,23  1 246 038   
12     13         La Libertad          Trujillo        25 499,9  1 778 080   
13     14          Lambayeque          Chiclayo        14 231,3  1 197 260   
14     15  Lima Metropolitana              Lima          267228  9 612 705   
15     16                Lima            Huacho       34 801,59    910 431   
16     17              Loreto           Iquitos      368 851,95    883 510   
17     18       Madre de Dios  Puerto Maldonado       85 300,54    141 070   
18     19            Moquegua          Moquegua       15 733,97    174 863   
19     20               Pasco    Cerro de Pasco       25 319,59    254 065   
20     21               Piura             Piura       35 892,49  1 856 809   
21     22                Puno              Puno       71 999,00  1 172 697   
22     23          San Martín         Moyobamba       51 253,31    813 381   
23     24               Tacna             Tacna       16 075,89    329 332   
24     25              Tumbes            Tumbes          466920    224 863   
25     26             Ucayali          Pucallpa      102 410,55    496 459   

                 Densidad2017 (hab/km²) Ubicación  
   Estimado 2020 Densidad2017 (hab/km²) Ubicación  
0        426 806                   9.66       NaN  
1      1 180 638                  30.17       NaN  
2        430 736                  19.42       NaN  
3      1 497 438                  21.83       NaN  
4        668 213                  14.06       NaN  
5      1 453 711                  40.25       NaN  
6      1 129 854                6766.19       NaN  
7      1 357 075                  16.75       NaN  
8        365 317                  15.71       NaN  
9        760 267                  19.57       NaN  
10       975 182                  39.89       NaN  
11     1 361 467                  28.19       NaN  
12     2 016 771                  69.73       NaN  
13     1 310 785                  69.73       NaN  
14     9 674 755                3208.86       NaN  
15     1 015 765                 272.56       NaN  
16     1 027 559                   2.40       NaN  
17       173 811                   1.65       NaN  
18       192 740                  11.11       NaN  
19       271 904                  10.03       NaN  
20     2 047 954                  51.73       NaN  
21     1 237 997                  16.28       NaN  
22       899 648                  15.87       NaN  
23       370 974                  20.49       NaN  
24       251 521                  48.16       NaN  
25       589 110                   4.85       NaN

Now, upon closer inspection, we notice that there are 26 "departamento"s and not 25. This is because the city of Lima, "Lima Metropolitana" was considered as an independent entity in the aformentioned wikipedia table due to its relevance.

However, we want Lima as a whole. In other words, sum the populations of "Lima Metropolitana" (city) and "Lima" (rest of the cities that are part of Lima "departamento"). We can deal with such problem as follows:

- First, we retrieve "Población" (population) column (estimates for 2020) and store it in pop_array `array`.
- Second, "Lima Metropolitana" and "Lima" are represented by indexes 14 and 15 respectively, so, in pop_array `array`, we add the value at index 15 to the value at index 14 and drop the value at index 15.

Now, pop_array `array` contains the population of each "departamento" in alphabetical order, which is going to match with our main dataframe, final_df, and the GeoJSON dataset that we are going to import in a few moments.

In [46]:
pop_array=pop_df[("Población","Estimado 2020")].apply(lambda x: int(x.replace("\xa0",""))).values
pop_array[14]=pop_array[14]+pop_array[15]
pop_array=np.delete(pop_array,15)

Then, we divide each column in "final_df" `dataframe` by "pop_array" `array`, multiply the result by 1000000 and replace "final_df"'s values. In other words, we go from "deaths" to "deaths per by million habitants".

In [49]:

for date in final_df.columns:
  final_df[date]=(final_df[date]/pop_array)*np.full([len(pop_array)],1000000)

Now, we proceed to import the GeoJSON data for every "departamento" of Peru and store it in "df_peru" `dataframe`.

In [50]:
df_peru = gpd.read_file('https://raw.githubusercontent.com/juaneladio/peru-geojson/master/peru_departamental_simple.geojson')

Then, we create a new column "coords" which is going to be used to porperly label each "departamento" with its name on the map. 

In [51]:
df_peru['coords'] = df_peru['geometry'].apply(lambda x: x.representative_point().coords[:])
df_peru['coords'] = [coords[0] for coords in df_peru['coords']]

Having finshed the data manipulation part, we can move onto plotting our choropleth maps.

- First, we set vmin and vmax variables to store the min and max global amount of deaths.
> If you don’t set this beforehand, Matplotlib will change the range of the choropleth each time the for loop iterates, so it will be harder to see how values have increased or decreased over time.
- Then, we create a `for loop` that, for each month, appends to df_peru `dataframe` (GeoJSON data) the column corresponding to that month of final_df `dataframe`, plots the choropleth map and then removes that column back so as not to increase the size of df_peru `dataframe` in each loop.
- Also, in each loop, we store the plotted map inside the just created "img" directory with padding zeros to keep a proper order.

This whole process will create a total of 27 choropleth maps (one per month) inside "img" directory .


In [74]:
if not os.path.exists("img"):
  os.mkdir("img")
title="Deaths per 'departamento', adjusted by population"
vmin, vmax=0,final_df.max().max()
df.columns
i=1
for month in final_df.columns.values:
  df_peru[month]=final_df[month].values
  fig, ax = plt.subplots(1, figsize=(13, 15))

  df_peru.plot(column=month,cmap='cool',
  linewidth=1, ax=ax,edgecolor='1', vmin=vmin, vmax=vmax,legend=True,
  norm=plt.Normalize(vmin=vmin, vmax=vmax))
  ax.axis("off")
  ax.set_title(title,fontsize=20)
  for idx, row in df_peru.iterrows():
    ax.text(row.coords[0], row.coords[1], row["NOMBDEP"], 
    horizontalalignment='center', 
    bbox={'facecolor': 'white', 'alpha':0.8, 'pad': 2, 'edgecolor':'none'})
  ax.annotate(f"{month}", xy=(0.2, .3), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='bottom',
            fontsize=30)
  ax.annotate("Deaths per million habitants", xy=(0.77, 0.57), xycoords='figure fraction',
            horizontalalignment='right', verticalalignment='top',
            fontsize=15, style='italic')
  if i<10:
    filepath = f"img/00{i}.jpg"
  else:
    filepath = f"img/0{i}.jpg"

  chart = ax.get_figure()
  chart.savefig(filepath, dpi=200)
  plt.close()
  df_peru.drop(columns=month, inplace=True)
  print(f"{i} of {len(final_df.columns)} processed")
  i+=1

1 of 27 processed
2 of 27 processed
3 of 27 processed
4 of 27 processed
5 of 27 processed
6 of 27 processed
7 of 27 processed
8 of 27 processed
9 of 27 processed
10 of 27 processed
11 of 27 processed
12 of 27 processed
13 of 27 processed
14 of 27 processed
15 of 27 processed
16 of 27 processed
17 of 27 processed
18 of 27 processed
19 of 27 processed
20 of 27 processed
21 of 27 processed
22 of 27 processed
23 of 27 processed
24 of 27 processed
25 of 27 processed
26 of 27 processed
27 of 27 processed


Last but not least, we make a .gif out of those 27 maps and, if preferred, erase them to save disk space.

Please note that in order to perform this action, you must install ImageMagick, as in `brew install imagemagick` (MacOS).

In [75]:
back=os.getcwd()
os.chdir("img")
subprocess.call([
  "convert", "-delay", "90", "-loop", "0", "*.jpg","output.gif"
])
for file_name in glob.glob("*.jpg"):
    os.remove(file_name)
os.chdir(back)

Optionally, we can also create a .mp4 video instead of a .gif.

Please note that in order to perform this action, you must install FFmpeg, as in `brew install ffmpeg` (MacOS).

In [ ]:
# back=os.getcwd()
# os.chdir("img")
# subprocess.call([
#     'ffmpeg', '-framerate', '24', '-i','%03d.jpg', '-r', '30',"-crf", "0", "-vcodec", "mpeg4", "-vf", "setpts=10*PTS",
#     'video_name.mp4'
# ])
# for file_name in glob.glob("*.jpg"):
#     os.remove(file_name)
# os.chdir(back)

Final output:
![maps](\img/output.gif\ "maps")

### References:

- https://towardsdatascience.com/how-to-make-a-gif-map-using-python-geopandas-and-matplotlib-cd8827cefbc8
- https://stackoverflow.com/questions/38899190/geopandas-label-polygons